# Object Tracking Notebook
This notebook combines approaches from Lab2_A, Lab2_C, and Lab2_E to perform object tracking instead of voxel tracking.

## 1. Import Required Libraries
Import all necessary libraries and utility functions.

In [2]:
# Dependencies and Imports
# Install required packages if not already installed
!pip install segment-anything torch torchvision clip-by-openai pillow matplotlib

import os
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import torch

# SAM and CLIP imports
from segment_anything import SamPredictor, sam_model_registry
import clip

# Utility imports (if available)
from lab_utils.data_utils import *
from lab_utils.detection_utils import *
from lab_utils.visualization_utils import *


  Using cached torch-2.8.0-cp313-none-macosx_11_0_arm64.whl.metadata (30 kB)
  Using cached torch-2.8.0-cp313-none-macosx_11_0_arm64.whl.metadata (30 kB)
  Using cached torchvision-0.23.0-cp313-cp313-macosx_11_0_arm64.whl.metadata (6.1 kB)
  Using cached torchvision-0.23.0-cp313-cp313-macosx_11_0_arm64.whl.metadata (6.1 kB)
  Using cached pillow-11.3.0-cp313-cp313-macosx_11_0_arm64.whl.metadata (9.0 kB)
  Using cached pillow-11.3.0-cp313-cp313-macosx_11_0_arm64.whl.metadata (9.0 kB)
  Using cached setuptools-80.9.0-py3-none-any.whl.metadata (6.6 kB)
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached setuptools-80.9.0-py3-none-any.whl.metadata (6.6 kB)
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached networkx-3.5-py3-none-any.whl.metadata (6.3 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached networkx-3.5-py3-none-any.whl.metadata (6.3 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)


ModuleNotFoundError: No module named 'numpy'

## 2. Load Data
Load scene data, images, and annotations.

In [ ]:
# Load data (update paths and logic as needed)
import os
scene_id = '40753679'
data_path = f'ARKitScenesData/{scene_id}/40753679_frames/lowres_wide/'

# Get all image file paths (assuming .png images)
images = [os.path.join(data_path, fname) for fname in os.listdir(data_path) if fname.endswith('.png')]
images.sort()  # Ensure temporal order

# If you have annotation loading logic, add it here
# annotations = load_annotations(data_path)


In [ ]:
# Test: Check loaded images
print(f"Loaded {len(images)} images.")
print("First 3 image paths:", images[:3])

## 3. Object Detection
Detect objects in the scene using methods from previous labs.

In [ ]:
# 3.1 Load SAM and CLIP models
def load_sam_clip(sam_checkpoint_path):
    from segment_anything import SamPredictor, sam_model_registry
    import clip
    import torch
    sam = sam_model_registry["vit_h"](checkpoint=sam_checkpoint_path)
    predictor = SamPredictor(sam)
    device = "cuda" if torch.cuda.is_available() else "cpu"
    clip_model, preprocess = clip.load("ViT-B/32", device=device)
    return predictor, clip_model, preprocess, device

sam_checkpoint_path = "path/to/sam_checkpoint.pth"
predictor, clip_model, preprocess, device = load_sam_clip(sam_checkpoint_path)


In [ ]:
# Test: Check SAM and CLIP model loading
print("Predictor type:", type(predictor))
print("CLIP model type:", type(clip_model))
print("Preprocess type:", type(preprocess))
print("Device:", device)

In [ ]:
# 3.2 Detect objects in images using SAM + CLIP
def detect_objects_sam_clip(images, predictor, clip_model, preprocess, device, text_prompts):
    import numpy as np
    from PIL import Image
    import torch
    detected_objects = []
    text_tokens = clip.tokenize(text_prompts).to(device)
    for img_path in images:
        image = np.array(Image.open(img_path))
        predictor.set_image(image)
        masks, _, _ = predictor.predict(
            point_coords=None,
            point_labels=None,
            multimask_output=True
        )
        for mask in masks:
            y_indices, x_indices = np.where(mask)
            if len(y_indices) == 0 or len(x_indices) == 0:
                continue
            y_min, y_max = y_indices.min(), y_indices.max()
            x_min, x_max = x_indices.min(), x_indices.max()
            crop = image[y_min:y_max, x_min:x_max]
            pil_crop = Image.fromarray(crop)
            clip_input = preprocess(pil_crop).unsqueeze(0).to(device)
            with torch.no_grad():
                image_features = clip_model.encode_image(clip_input)
                text_features = clip_model.encode_text(text_tokens)
                logits_per_image = (100.0 * image_features @ text_features.T).softmax(dim=-1)
                pred_class = text_prompts[logits_per_image.argmax().item()]
            detected_objects.append({
                "image": img_path,
                "bbox": [x_min, y_min, x_max, y_max],
                "mask": mask,
                "class": pred_class
            })
    return detected_objects

# Example usage:
text_prompts = ["sofa", "shelf", "table", "chair"]
detected_objects = detect_objects_sam_clip(images, predictor, clip_model, preprocess, device, text_prompts)


In [ ]:
# Test: Check detection output
print(f"Detected {len(detected_objects)} objects.")
if detected_objects:
    print("First detected object:", detected_objects[0])

## 4. Object Tracking Logic
Implement object tracking across frames.

In [ ]:
# 4.1 Define object tracking function (IoU-based, no voxels)
def track_objects(detected_objects, iou_threshold=0.5):
    """
    Track objects across frames using IoU and class consistency.
    """
    # Sort detected objects by image path to ensure temporal order
    detected_objects.sort(key=lambda x: x["image"])
    # Group detections by frame
    frames = {}
    for obj in detected_objects:
        img_path = obj["image"]
        if img_path not in frames:
            frames[img_path] = []
        frames[img_path].append(obj)
    frame_list = sorted(frames.keys())
    detections_by_frame = [frames[f] for f in frame_list]
    next_track_id = 0
    tracked_objects = []
    # First frame: assign new track IDs
    if detections_by_frame:
        for detection in detections_by_frame[0]:
            detection["track_id"] = next_track_id
            tracked_objects.append(detection)
            next_track_id += 1
    # Subsequent frames: associate detections
    for frame_idx in range(1, len(detections_by_frame)):
        current_detections = detections_by_frame[frame_idx]
        prev_frame_tracks = [obj for obj in tracked_objects if obj["image"] == frame_list[frame_idx-1]]
        for detection in current_detections:
            best_iou = iou_threshold
            best_match = -1
            for track in prev_frame_tracks:
                if track["class"] != detection["class"]:
                    continue
                # IoU calculation
                det_bbox = detection["bbox"]
                track_bbox = track["bbox"]
                x1 = max(det_bbox[0], track_bbox[0])
                y1 = max(det_bbox[1], track_bbox[1])
                x2 = min(det_bbox[2], track_bbox[2])
                y2 = min(det_bbox[3], track_bbox[3])
                if x2 < x1 or y2 < y1:
                    iou = 0
                else:
                    intersection = (x2 - x1) * (y2 - y1)
                    det_area = (det_bbox[2] - det_bbox[0]) * (det_bbox[3] - det_bbox[1])
                    track_area = (track_bbox[2] - track_bbox[0]) * (track_bbox[3] - track_bbox[1])
                    union = det_area + track_area - intersection
                    iou = intersection / union if union > 0 else 0
                if iou > best_iou:
                    best_iou = iou
                    best_match = track["track_id"]
            if best_match >= 0:
                detection["track_id"] = best_match
            else:
                detection["track_id"] = next_track_id
                next_track_id += 1
            tracked_objects.append(detection)
    return tracked_objects

# 4.2 Apply tracking to detected objects
tracked_objects = track_objects(detected_objects)


In [ ]:
# Test: Check tracking output
print(f"Tracked {len(tracked_objects)} objects.")
if tracked_objects:
    print("First tracked object:", tracked_objects[0])

## 5. Visualization
Visualize tracked objects over time.

In [ ]:
# Visualize tracking results
# visualize_tracking(tracked_objects, images)

## 6. Evaluation
Evaluate tracking performance using ground truth.

In [ ]:
# Evaluate tracking
# evaluation_metrics = evaluate_tracking(tracked_objects, ground_truth)